# fse - Tutorial

Welcome to fse - fast sentence embeddings. The library is intended to compute sentence embeddings as fast as possible. 
It offers a simple and easy to understand syntax for you to use in your own projects. Before we start with any model, lets have a look at the input types.
All fse models require an iterable/generator which produces a tuple. The tuple has two fields: words and index. The index is required for the multi-thread processing, as sentences might not be processed sequentially. The index dictates, which row of the corresponding sentence vector matrix the sentence belongs to.

## Input handling

In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
s = (["Hello", "world"], 0)
print(s[0])
print(s[1])

['Hello', 'world']
0


The words of the tuple will always consist of a list of strings. Otherwise the train method will raise an Error. However, most input data is available as a list of strings.

In order to deal with this common input format, fse provides the IndexedList and some variants, which handel all required data operations for you. You can provide multiple lists (or sets) which will all be merged into a single list. This eases work if you have to work with the STS datasets.

The multiple types of indexed lists. Let's go through them one by one:
- IndexedList: for already pre-splitted sentences
- **C**IndexedList: for already pre-splitted sentences with a custom index for each sentence
- SplitIndexedList: for sentences which have not been splitted. Will split the strings
- Split**C**IndexedList: for sentences which have not been splitted and with a custom index for each sentence
- **C**SplitIndexedList: for sentences which have not been splitted. Will split the strings. You can provide a custom split function
- **C**Split*C*IndexedList: for sentences where you want to provide a custom index and a custom split function.

*Note*: These are ordered by speed. Meaning, that IndexedList is the fastest, while **C**Split*C*IndexedList is the slowest variant.

In [3]:
from fse import SplitIndexedList

sentences_a = ["Hello there", "how are you?"]
sentences_b = ["today is a good day", "Lorem ipsum"]

s = SplitIndexedList(sentences_a, sentences_b)
print(len(s))
s[0]

4


(['Hello', 'there'], 0)

To save memory, we do not convert the original lists inplace. The conversion will only take place once you call the getitem method. To access the original data, call:

In [4]:
s.items

['Hello there', 'how are you?', 'today is a good day', 'Lorem ipsum']

If the data is already preprocessed as a list of lists you can just use the IndexedList

In [5]:
from fse import IndexedList

sentences_splitted = ["Hello there".split(), "how are you?".split()]
s = IndexedList(sentences_splitted)
print(len(s))
s[0]

2


(['Hello', 'there'], 0)

In case you want to provide your own splitting function, you can pass a callable to the **C**SplitIndexedList class.

In [6]:
from fse import CSplitIndexedList

def split_func(string):
    return string.lower().split()

s = CSplitIndexedList(sentences_a, custom_split=split_func)
print(len(s))
s[0]

2


(['hello', 'there'], 0)

If you want to stream a file from disk (where each line corresponds to a sentence) you can use the IndexedLineDocument.

In [7]:
from fse import IndexedLineDocument
doc = IndexedLineDocument("../test/test_data/test_sentences.txt")

In [8]:
i = 0
for s in doc:
    print(f"{s[1]}\t{s[0]}")
    i += 1
    if i == 4:
        break

0	['Good', 'stuff', 'i', 'just', 'wish', 'it', 'lasted', 'longer']
1	['Hp', 'makes', 'qualilty', 'stuff']
2	['I', 'like', 'it']
3	['Try', 'it', 'you', 'will', 'like', 'it']


If you are later working with the similarity of sentences, the IndexedLineDocument provides you the option to access each line by its corresponding index. This helps you in determining the similarity of sentences, as the most_similar method would otherwise just return indices.

In [9]:
doc[20]

'I feel like i just got screwed'

# Training a model / Performing inference

Training a fse model is simple. You only need a pre-trained word embedding model which you use during the initializiation of the fse model you want to use.

In [10]:
from fse import Vectors
import gensim.downloader as api
data = api.load("quora-duplicate-questions")
glove = Vectors.from_pretrained("glove-wiki-gigaword-100")

2022-04-10 21:00:42,265 : MainThread : INFO : Lock 23311648346896 acquired on /home/oborchers/.cache/huggingface/hub/fse--glove-wiki-gigaword-100.main.3282d5e7c5e979c2411ba9703d63a46243a2047e/glove-wiki-gigaword-100.model.lock
2022-04-10 21:00:42,267 : MainThread : INFO : Lock 23311648346896 released on /home/oborchers/.cache/huggingface/hub/fse--glove-wiki-gigaword-100.main.3282d5e7c5e979c2411ba9703d63a46243a2047e/glove-wiki-gigaword-100.model.lock
2022-04-10 21:00:47,947 : MainThread : INFO : Lock 23311648386016 acquired on /home/oborchers/.cache/huggingface/hub/fse--glove-wiki-gigaword-100.main.3282d5e7c5e979c2411ba9703d63a46243a2047e/glove-wiki-gigaword-100.model.vectors.npy.lock
2022-04-10 21:00:47,949 : MainThread : INFO : Lock 23311648386016 released on /home/oborchers/.cache/huggingface/hub/fse--glove-wiki-gigaword-100.main.3282d5e7c5e979c2411ba9703d63a46243a2047e/glove-wiki-gigaword-100.model.vectors.npy.lock
2022-04-10 21:00:47,951 : MainThread : INFO : loading Vectors object

In [11]:
sentences = []
for d in data:
    # Let's blow up the data a bit by replicating each sentence.
    for i in range(8):
        sentences.append(d["question1"].split())
        sentences.append(d["question2"].split())
s = IndexedList(sentences)
print(len(s))

6468640


So we have about 6468640 sentences we want to compute the embeddings for. If you import the FAST_VERSION variable as follows you can ensure, that the compiliation of the cython routines worked correctly:

In [12]:
from fse.models.average import FAST_VERSION, MAX_WORDS_IN_BATCH
print(MAX_WORDS_IN_BATCH)
print(FAST_VERSION)
# 1 -> The fast version works

10000
1


In [13]:
from fse.models import uSIF
model = uSIF(glove, workers=1, lang_freq="en")

2022-04-10 21:01:20,427 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en


In [14]:
model.train(s)

2022-04-10 21:01:21,811 : MainThread : INFO : scanning all indexed sentences and their word counts
2022-04-10 21:01:26,812 : MainThread : INFO : SCANNING : finished 4268552 sentences with 47204452 words
2022-04-10 21:01:29,815 : MainThread : WARNING : found 16 empty sentences
2022-04-10 21:01:29,816 : MainThread : INFO : finished scanning 6468640 sentences with an average length of 11 and 71556728 total words
2022-04-10 21:01:30,127 : MainThread : INFO : estimated memory for 6468640 sentences with 100 dimensions and 400000 vocabulary: 2621 MB (2 GB)
2022-04-10 21:01:30,129 : MainThread : INFO : initializing sentence vectors for 6468640 sentences
2022-04-10 21:01:58,270 : MainThread : INFO : pre-computing uSIF weights for 400000 words
2022-04-10 21:01:59,606 : MainThread : INFO : begin training
2022-04-10 21:02:04,625 : MainThread : INFO : PROGRESS : finished 20.57% with 1330528 sentences and 10112720 words, 266105 sentences/s
2022-04-10 21:02:09,626 : MainThread : INFO : PROGRESS : fin

(6468624, 49255184)

The models training speed revolves at around 300,000 - 500,000 sentences / seconds. That means we finish the task in about 10 seconds. This is **heavily dependent** on the input processing. If you train ram-to-ram it is naturally faster than any ram-to-disk or disk-to-disk varianty. Similarly, the speed depends on the workers.

Once the sif model is trained, you can perform additional inferences for unknown sentences. This two step process for new data is required, as computing the principal components for models like SIF and uSIF will require a fair amount of sentences. If you want the vector for a single sentence (which is out of the training vocab), just use:

In [15]:
tmp = ("Hello my friends".split(), 0)
model.infer([tmp])

2022-04-10 21:02:44,544 : MainThread : INFO : scanning all indexed sentences and their word counts
2022-04-10 21:02:44,545 : MainThread : INFO : finished scanning 1 sentences with an average length of 3 and 3 total words
2022-04-10 21:02:44,546 : MainThread : INFO : removing 5 principal components took 0s


array([[ 2.58816600e-01, -2.95302048e-02,  2.75351852e-02,
        -2.84114152e-01, -7.67392814e-02,  4.69468027e-01,
        -1.08480439e-01,  2.74309274e-02, -6.55729175e-02,
        -3.49144369e-01, -3.67706642e-03, -7.37217665e-02,
         1.99556813e-01,  1.58444345e-01, -1.19665325e-01,
        -2.94139415e-01,  9.47405249e-02, -1.60569757e-01,
        -3.78268361e-01,  3.62190604e-01, -1.04472280e-01,
         2.73262203e-01, -3.63826789e-02, -1.77271560e-01,
         1.13316491e-01,  9.37681645e-02, -2.24020645e-01,
        -5.91988862e-02,  4.76207793e-01,  1.19424753e-01,
         2.50848651e-01,  3.00246894e-01,  3.93525660e-01,
         1.26845583e-01,  9.48224217e-04,  2.53242493e-01,
         1.82835817e-01,  6.30666614e-02,  2.79401392e-01,
        -1.32174164e-01, -1.33375764e-01,  1.35822147e-01,
         2.27280006e-01, -1.15767486e-01, -1.42402038e-01,
        -1.17349595e-01, -4.09546375e-01,  3.27205539e-01,
         4.02800381e-01, -1.04363769e-01, -1.11321002e-0

## Querying the model

In order to query the model or perform similarity computations we can just access the model.sv (sentence vectors) object and use its method. To get a vector for an index, just call

In [16]:
model.sv[0]

array([ 0.06577122,  0.00125362,  0.02861076,  0.29774222,  0.16603038,
       -0.3326527 , -0.25274056, -0.11763255, -0.00691745, -0.09893274,
       -0.03162983,  0.11619435, -0.06521351, -0.2655796 , -0.19060446,
       -0.0539908 , -0.00769202,  0.06885006,  0.02343568,  0.1532896 ,
        0.09342796,  0.04696935, -0.23081023,  0.1498743 , -0.14505012,
        0.02421208,  0.05296347,  0.06859872, -0.07599732,  0.00725658,
       -0.24535023,  0.22700995, -0.09825065, -0.09442319,  0.13708887,
        0.15521362,  0.20619243, -0.10562573, -0.09422812, -0.21555066,
       -0.04667541, -0.0792224 ,  0.03256182, -0.09426205, -0.24983093,
        0.13597786, -0.2398801 ,  0.05768704, -0.06787113, -0.4941577 ,
       -0.22496416, -0.00669573, -0.03549183,  0.29734784, -0.17858498,
       -0.5486123 , -0.14864792, -0.03553995,  0.67317814, -0.076203  ,
        0.05146242, -0.18583792, -0.15082437, -0.00156193,  0.05055934,
        0.14105804, -0.19593558,  0.21223646,  0.12954028, -0.07

To compute the similarity or distance between two sentence from the training set you can call:

In [17]:
print(model.sv.similarity(0,1).round(3))
print(model.sv.distance(0,1).round(3))

0.965
0.035


We can further call for the most similar sentences given an index. For example, we want to know the most similar sentences for sentence index 100:

In [18]:
print(s[100])

(['Should', 'I', 'buy', 'tiago?'], 100)


In [19]:
model.sv.most_similar(100)
# Division by zero can happen if you encounter empy sentences

2022-04-10 21:02:46,966 : MainThread : INFO : precomputing L2-norms of sentence vectors


[(5742433, 1.0),
 (5172303, 1.0),
 (1657838, 1.0),
 (5742447, 1.0),
 (5742445, 1.0),
 (5742443, 1.0),
 (5742441, 1.0),
 (5742439, 1.0),
 (5742437, 1.0),
 (5742435, 1.0)]

However, the preceding function will only supply the indices of the most similar sentences. You can circumvent this problem by passing an indexable function to the most_similar call:

In [20]:
model.sv.most_similar(100, indexable=s.items)

[(['Should', 'I', 'buy', 'Google', 'Glass?'], 5742433, 1.0),
 (['Why', "doesn't", 'Apple', 'buy', 'Samsung?'], 5172303, 1.0),
 (['Should', 'I', 'buy', 'Moto', 'G4', 'Plus?'], 1657838, 1.0),
 (['Should', 'I', 'buy', 'Google', 'Glass?'], 5742447, 1.0),
 (['Should', 'I', 'buy', 'Google', 'Glass?'], 5742445, 1.0),
 (['Should', 'I', 'buy', 'Google', 'Glass?'], 5742443, 1.0),
 (['Should', 'I', 'buy', 'Google', 'Glass?'], 5742441, 1.0),
 (['Should', 'I', 'buy', 'Google', 'Glass?'], 5742439, 1.0),
 (['Should', 'I', 'buy', 'Google', 'Glass?'], 5742437, 1.0),
 (['Should', 'I', 'buy', 'Google', 'Glass?'], 5742435, 1.0)]

There we go. This is a lot more understandable than the initial list of indices.

To search for sentences, which are similar to a given word vector, you can call:

In [21]:
model.sv.similar_by_word("easy", wv=glove, indexable=s.items)

[(['Which',
   'is',
   'more',
   'easy',
   'to',
   'learn?',
   'Ruby',
   'on',
   'Rails',
   'or',
   'Python/Django?'],
  4717063,
  0.9466769099235535),
 (['Which',
   'is',
   'more',
   'easy',
   'to',
   'learn?',
   'Ruby',
   'on',
   'Rails',
   'or',
   'Python/Django?'],
  4717065,
  0.9466769099235535),
 (['Which',
   'is',
   'more',
   'easy',
   'to',
   'learn?',
   'Ruby',
   'on',
   'Rails',
   'or',
   'Python/Django?'],
  4717061,
  0.9466769099235535),
 (['Which',
   'is',
   'more',
   'easy',
   'to',
   'learn?',
   'Ruby',
   'on',
   'Rails',
   'or',
   'Python/Django?'],
  4717057,
  0.9466769099235535),
 (['Which',
   'is',
   'more',
   'easy',
   'to',
   'learn?',
   'Ruby',
   'on',
   'Rails',
   'or',
   'Python/Django?'],
  4717067,
  0.9466769099235535),
 (['Which',
   'is',
   'more',
   'easy',
   'to',
   'learn?',
   'Ruby',
   'on',
   'Rails',
   'or',
   'Python/Django?'],
  4717069,
  0.9466769099235535),
 (['Which',
   'is',
   'mor

Furthermore, you can query for unknown (or new) sentences by calling:

In [22]:
model.sv.similar_by_sentence("Is this really easy to learn".split(), model=model, indexable=s.items)

2022-04-10 21:02:52,537 : MainThread : INFO : scanning all indexed sentences and their word counts
2022-04-10 21:02:52,537 : MainThread : INFO : finished scanning 1 sentences with an average length of 6 and 6 total words
2022-04-10 21:02:52,538 : MainThread : INFO : removing 5 principal components took 0s


[(['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  418230,
  0.9860147833824158),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  418232,
  0.9860147833824158),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  418236,
  0.9860147833824158),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  6255670,
  0.9860147833824158),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  418226,
  0.9860147833824158),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  418228,
  0.9860147833824158),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  418238,
  0.9860147833824158),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  6255666,
  0.9860147833824158),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  418234,
  0.9860147833824158),
 (['How', 'do', 'I', 'learn', 'Python', 'in', 'easy', 'way?'],
  6255674,
  0.9860147833824158)]

Feel free to browse through the library and get to know the functions a little better!